In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Iniciamos con la lectura de los datos

In [96]:
df = pd.read_excel("real_estate_data.xlsx", index_col = [0])

#### Formateo de los nombres de las columnas para remover espacios

In [97]:
def format_cols_spaces(columns):
  return {key:key.replace(" ","_") for key in columns}

In [98]:
columns_replace = format_cols_spaces(df.columns)
df = df.rename(columns = columns_replace)
df_train = df[0:315]
df_test = df[315:]

### Creamos la clase NeuralNetwork

In [287]:
class NeuralNetwork:
  def __init__(self):

    #Inicializacion de pesos
    # set seed
    np.random.seed(42)
    self.W1 = np.random.random((5,6))
    self.b1 = np.random.random((5,1))

    self.W2 = np.random.random((1,5))
    self.b2 = np.random.random((1,1))

  def relu(self,X):
    return np.maximum(0,X)

  def sigmoid(self,X):
    return 1/(1+np.exp(-X))

  def forward(self, X):
    zi = self.W1@X + self.b1
    zi = self.sigmoid(zi)
    y_hat = self.W2@zi + self.b2
    return y_hat.T # y_hat.T es un vector columna
  
  def funcion_objetivo(self, X, y):
    return 0.5*np.mean((self.forward(X) - y)**2)
  
  def numerical_gradient(self, x, y):
        # Calcular el gradiente numérico
        eps = 1e-4
        grads = {}
        for param_name in ["W1", "b1", "W2", "b2"]:
            param = getattr(self, param_name)
            grad = np.zeros_like(param)
            it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                original_value = param[ix]
                param[ix] = original_value + eps
                grad_plus = self.funcion_objetivo(x,y)
                param[ix] = original_value - eps
                grad_minus = self.funcion_objetivo(x,y)
                grad[ix] = (grad_plus - grad_minus) / (2 * eps)
                param[ix] = original_value
                it.iternext()
            grads[param_name] = grad
        return grads
  
#funcion fit y loop de entrenamiento
  def fit(self,X, y, learning_rate=0.001, epochs=10000):
      eps = 1e-3
      loss_accum = []
      for i in range(epochs):
            gradients = self.numerical_gradient(X, y)

            self.W1 -= learning_rate * gradients[0]
            self.b1 -= learning_rate * gradients[1]
            self.W2 -= learning_rate * gradients[2]
            self.b2 -= learning_rate * gradients[3]

            loss = self.funcion_objetivo(X, y)
            loss_accum.append(loss)

            print(f"Epoch {i+1}/{epochs} - Loss: {loss}")

      return loss_accum



In [288]:
neural_1 = NeuralNetwork()

In [101]:
df_train_X = df_train.drop(['Y_house_price_of_unit_area'], axis = 1)
df_train_y = df_train[['Y_house_price_of_unit_area']]

In [279]:
train_X_values = df_train_X.values.T
train_y_values = df_train_y.values


In [291]:
train_x_shorter = train_X_values[:,0:10]
train_y_shorter = df_train_y.values[0:10]

In [289]:
neural_1.numerical_gradient(train_X_values, train_y_values)

{'W1': array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 'W2': array([[-35.65209936, -35.65209936, -35.65209936, -35.65209936,
         -35.65209936]]),
 'b2': array([[-35.65209936]])}

In [293]:
neural_1.fit(train_x_shorter, train_y_shorter, 1000, 0.01)

Epoch: 0, Loss: nan
Epoch: 100, Loss: nan
Epoch: 200, Loss: nan
Epoch: 300, Loss: nan
Epoch: 400, Loss: nan
Epoch: 500, Loss: nan
Epoch: 600, Loss: nan
Epoch: 700, Loss: nan
Epoch: 800, Loss: nan
Epoch: 900, Loss: nan


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
def predict(x):
        y =
        return y